In [39]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path

import transformers 

# load PDF file

In [40]:
link_to_pdf = "./vqa.pdf"

In [41]:
loader = PyPDFLoader(link_to_pdf)
documents = loader.load_and_split()
len(documents)

39

# load embedding and model

In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=64
)
texts = text_splitter.split_documents(documents)
len(texts)

128

In [ ]:
``

In [19]:

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

loading configuration file C:\Users\joybo/.cache\torch\sentence_transformers\sentence-transformers_all-MiniLM-L6-v2\config.json
Model config BertConfig {
  "_name_or_path": "C:\\Users\\joybo/.cache\\torch\\sentence_transformers\\sentence-transformers_all-MiniLM-L6-v2\\",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file C:\Users\joybo/.cache\torch\sentence_transformers\sentence-transformers_all-MiniLM-L6-v2\pytorch_model.bin
All model ch

In [44]:
model_n_ctx = 1000
model_path = "ggml-vicuna-13b-1.1-q4_2.bin"
llm = GPT4All(model=model_path, n_ctx=1000, backend="gptj", verbose=False)

Exception ignored in: <function Model.__del__ at 0x0000020CD218B4C0>
Traceback (most recent call last):
  File "c:\Python311\Lib\site-packages\pyllamacpp\model.py", line 402, in __del__
    if self._ctx:
       ^^^^^^^^^
AttributeError: 'Model' object has no attribute '_ctx'


ValidationError: 2 validation errors for GPT4All
backend
  extra fields not permitted (type=value_error.extra)
__root__
  Model.__init__() got an unexpected keyword argument 'ggml_model' (type=type_error)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
)

# Interact with PDF file |

In [ ]:
res = qa(f"""
    How much is the dividend per share during during 2022?
    Extract it from the text.
""")
print(res["result"])